# igraph

This Jupyter notebook provides an example of using the Python packages [gravis](https://pypi.org/project/gravis) and [igraph](https://igraph.org). The .ipynb file can be found [here](https://github.com/robert-haas/gravis/tree/master/examples).

## References

- [igraph website](https://igraph.org)
  - [Documentation](https://igraph.org/python/#docs) for the Python interface
    - [Tutorial](https://igraph.org/python/doc/tutorial/tutorial.html)
    - [API reference](https://igraph.org/python/api/latest/)

## Installation

- With [pip](https://pypi.org/project/igraph): `pip install igraph`
- With [conda](https://anaconda.org/search?q=igraph): `conda install -c conda-forge igraph`

## Import

In [ ]:
import os

import igraph as ig

import gravis as gv

## Quick start

In [ ]:
def assign_properties(g):
    # Centrality calculation
    node_centralities = g.betweenness()
    edge_centralities = g.edge_betweenness()
    
    # Community detection
    communities = g.community_fastgreedy().as_clustering().membership
    
    # Graph properties
    g['node_border_size'] = 1.5
    g['node_border_color'] = 'black'
    g['edge_opacity'] = 0.75

    # Node properties: Size by centrality, color by community
    colors = ['red', 'blue', 'green', 'orange', 'pink', 'brown', 'yellow', 'cyan', 'magenta', 'violet']
    g.vs['size'] = [10.0 + val / 50.0 for val in node_centralities]
    g.vs['color'] = [colors[community_index % len(colors)] for community_index in communities]
    
    # Edge properties: Size by centrality, color by community (within=community color, between=black)
    g.es['size'] = [0.5 + val / 100.0 for val in edge_centralities]
    g.es['color'] = [colors[communities[i] % len(colors)] if communities[i] == communities[j] else 'black'
                     for i, j in g.get_edgelist()]


# Create a graph with a generator function
g = ig.Graph.GRG(200, 0.14)

# Scale the coordinates provided by this particular graph generator (alternative: delete them)
g.vs['x'] = [val * 2000 - 1000 for val in g.vs['x']]  # del g.vs['x']
g.vs['y'] = [val * 2000 - 1000 for val in g.vs['y']]  # del g.vs['y']

# Assign properties
assign_properties(g)

# Plot it
gv.d3(g, zoom_factor=0.2)

## Graph construction

### 1) Manual graph construction

- Tutorial
    - [Creating a graph from scratch](https://igraph.org/python/doc/tutorial/tutorial.html#creating-a-graph-from-scratch)
- API reference
    - [Graph](https://igraph.org/python/api/latest/igraph.Graph.html#__init__)
        - [add_vertex](https://igraph.org/python/api/latest/igraph.Graph.html#add_vertex)
        - [add_vertices](https://igraph.org/python/api/latest/igraph.Graph.html#add_vertices)
        - [add_edge](https://igraph.org/python/api/latest/igraph.Graph.html#add_edge)
        - [add_edges](https://igraph.org/python/api/latest/igraph.Graph.html#add_edges)

#### 1a) Graph (with directed=False)

undirected, with self-loops, with parallel edges, with attributes

In [ ]:
ug = ig.Graph(directed=False)


# Node with automatic id (starts from 0)
ug.add_vertex()

# Node with user-defined id (=only a synonym, also gets an automatic id!)
ug.add_vertex(name='b')

# Node + attribute
ug.add_vertex(name='c', size=20)

# Node + attributes
ug.add_vertex(name='d', size=30, color='orange')


# Nodes
ug.add_vertices(2)           # argument: number of nodes with automatic ids
ug.add_vertices('g')         # argument: user-defined id (also gets an automatic id)
ug.add_vertices(['h', 'i'])  # argument: iterable of user-defined ids (also get automatic ids)


# Edge (nodes need to already exist)
ug.add_edge(0, 1)
ug.add_edge('b', 'c')

# Edge + attribute
ug.add_edge('c', 'd', size=3)

# Edge + attributes
ug.add_edge('d', 4, size=4, color='orange')


# Edges
ug.add_edges([
    (4, 5),
    (5, 'g'),
    ('g', 'h'),
    ('h', 'i'),
    (8, 0),
    (0, 0),
    (0, 0),
    (0, 1),
    (0, 1),
])


gv.d3(ug, graph_height=200)

#### 1b) Graph (with directed=True)

undirected, with self-loops, with parallel edges, with attributes

In [ ]:
dg = ig.Graph(directed=True, edges=[(e.source, e.target) for e in ug.es])

gv.d3(dg, graph_height=200)

#### Assign attributes to a created graph

In [ ]:
g = ig.Graph(directed=False, edges=[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 0)])

Graph attributes

In [ ]:
g['background_color'] = 'gray'
g['node_shape'] = 'rectangle'
g['node_label_color'] = 'white'
g['edge_opacity'] = 0.3

Node attributes

In [ ]:
# Nodes
num_nodes = len(g.vs)
g.vs['size'] = [5 + i*5 for i in range(num_nodes)]
g.vs['color'] = ['lightblue'] * num_nodes

# Node
g.vs[3]['color'] = 'darkred'
g.vs[3]['shape'] = 'hexagon'
g.vs[3]['size'] = 40
g.vs[3]['opacity'] = 0.3

Edge attributes

In [ ]:
# Edges
num_edges = len(g.es)
g.es['size'] = [1 + i for i in range(num_edges)]
g.es['color'] = ['lightgreen'] * num_edges

# Edge
g.es[3]['size'] = 1
g.es[3]['color'] = 'darkred'

In [ ]:
gv.d3(g, graph_height=200, use_centering_force=False)

### 2) Algorithmic graph construction

- Tutorial
    - [Generating graphs](https://igraph.org/python/doc/tutorial/tutorial.html#generating-graphs)
- API reference
  - [Barabasi](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#Barabasi)
  - [De Bruijn](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#De_Bruijn)
  - [Degree Sequence](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#Degree_Sequence)
  - [Erdos-Renyi](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#Erdos_Renyi)
  - [Establishment](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#Establishment)
  - [Forest Fire](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#Forest_Fire)
  - [Full](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#Full)
  - [Full Bipartite](https://igraph.org/python/api/latest/igraph.Graph.html#Full_Bipartite)
  - [Full Citation](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#Full_Citation)
  - [Growing Random](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#Growing_Random)
  - [GRG](https://igraph.org/python/api/latest/igraph.Graph.html#GRG)
  - [Isoclass](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#Isoclass)
  - [Kautz](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#Kautz)
  - [K_Regular](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#K_Regular)
  - [Lattice](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#Lattice)
  - [Lederberg-Coxeter-Frucht (LCF)](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#LCF)
  - [Preference](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#Preference)
  - [Random Bipartite](https://igraph.org/python/api/latest/igraph.Graph.html#Random_Bipartite)
  - [Realize Degree Sequence](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#Realize_Degree_Sequence)
  - [Recent_Degree](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#Recent_Degree)
  - [Ring](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#Ring)
  - [Star](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#Star)
  - [Static Fitness](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#Static_Fitness)
  - [Static Power Law](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#Static_Power_Law)
  - [Stochastic Block Model (SBM)](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#SBM)
  - [Tree](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#Tree)
  - [Tree Game](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#Tree_Game)
  - [Watts-Strogatz](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#Watts_Strogatz)

In [ ]:
n = 10

g = ig.Graph.Barabasi(10)
g = ig.Graph.Erdos_Renyi(50, 0.1)
g = ig.Graph.Watts_Strogatz(2, 3, 3, 0.1)

### 3) Graph loading from an internal collection

- API reference
  - [Atlas](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#Atlas)
  - [Famous](https://igraph.org/python/api/latest/igraph._igraph.GraphBase.html#Famous)

In [ ]:
g = ig.Graph.Atlas(22)
g = ig.Graph.Famous('Chvatal')

### 4) Graph import and export

- Tutorial
    - [igraph and the outside world](https://igraph.org/python/doc/tutorial/tutorial.html#igraph-and-the-outside-world)

#### Import

In [ ]:
# TODO

#### Export

In [ ]:
# TODO

## Basic graph inspection

### 1) Graph and its properties

In [ ]:
print('Type:', type(g))
print('Directed:', g.is_directed())

### 2) Nodes and their properties

In [ ]:
for node in g.vs:
    node_id = node.index
    attributes = node.attributes()
    degree = node.degree()
    print('Type:', type(node), type(attributes))
    print('Id:', node_id)
    print('Attributes:', attributes)
    print('Degree:', degree)
    break

### 3) Edges and their properties

In [ ]:
for edge in g.es:
    source = edge.source
    target = edge.target
    attributes = edge.attributes()
    print('Type:', type(source), type(target), type(attributes))
    print('Source:', source)
    print('Target:', target)
    print('Attributes:', )
    break

In [ ]:
edge_list = g.get_edgelist()
edge_list[0:3]

## Calculating graph measures and metrics

### 1) Quantitative measures

In [ ]:
g.degree()
g.betweenness()
g.edge_betweenness()
g.pagerank()

# TODO: more measures are available

### 2) Structure inference

#### Community detection and graph partitioning

- API reference
    - [community_edge_betweenness](https://igraph.org/python/api/latest/igraph.Graph.html#community_edge_betweenness)
    - [community_fastgreedy](https://igraph.org/python/api/latest/igraph.Graph.html#community_fastgreedy)
    - [community_infomap](https://igraph.org/python/api/latest/igraph.Graph.html#community_infomap)
    - [community_label_propagation](https://igraph.org/python/api/latest/igraph.Graph.html#community_label_propagation)
    - [community_leading_eigenvector](https://igraph.org/python/api/latest/igraph.Graph.html#community_leading_eigenvector)
    - [community_leading_eigenvector_naive](https://igraph.org/python/api/latest/igraph.Graph.html#community_leading_eigenvector_naive)
    - [community_multilevel](https://igraph.org/python/api/latest/igraph.Graph.html#community_multilevel)
    - [community_optimal_modularity](https://igraph.org/python/api/latest/igraph.Graph.html#community_optimal_modularity)
    - [community_spinglass](https://igraph.org/python/api/latest/igraph.Graph.html#community_spinglass)
    - [community_walktrap](https://igraph.org/python/api/latest/igraph.Graph.html#community_walktrap)
    - [modularity](https://igraph.org/python/api/latest/igraph.Graph.html#modularity)

In [ ]:
g = ig.Graph.GRG(40, 0.4)

# based on the betweenness of the edges in the network
g.community_edge_betweenness()

# Greedily maximize the modularity score of the graph - Clauset, Newman, Moore
g.community_fastgreedy()

# Infomap method of Rosvall and Bergstrom
g.community_infomap()

# Label propagation method of Raghavan, Albert, Kumara
g.community_label_propagation()

# Newman's leading eigenvector method
g.community_leading_eigenvector()

# g.community_leading_eigenvector_naive()

# Multilevel algorithm of Blondel et al.
g.community_multilevel()

# Calculates the optimal modularity score with GNU Linear Programming Kit
g.community_optimal_modularity()

# Spinglass community detection method of Reichardt and Bornholdt
g.community_spinglass()

# Community detection algorithm of Latapy and Pons, based on random walks
g.community_walktrap()

## Graph visualization

### Layout calculation

- Tutorial
    - [Layouts and plotting](https://igraph.org/python/doc/tutorial/tutorial.html#layouts-and-plotting)
- API reference
    - [Layout class](https://igraph.org/python/api/latest/igraph.layout.Layout.html)
    - [layout](https://igraph.org/python/api/latest/igraph.Graph.html#layout)
    - [drawing package](https://igraph.org/python/api/latest/igraph.drawing.html)

In [ ]:
# TODO

### Plot

In [ ]:
# TODO